# Transformation

In [ ]:
#| default_exp data_utils.transform

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2
from ipynb_path import *
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
#| export
from __future__ import annotations
from relax.data_utils.preprocessing import *
from relax.import_essentials import *

In [ ]:
#| hide
import sklearn.preprocessing as skp
from fastcore.test import test_fail
from copy import deepcopy

In [ ]:
#| export
class Transformation:
    def __init__(self, name, transformer: DataPreprocessor = None):
        self.name = name
        self.transformer = transformer

    @property
    def is_categorical(self) -> bool:
        if self.transformer is None:
            return False
        return isinstance(self.transformer, EncoderPreprocessor)

    def fit(self, xs, y=None):
        self.transformer.fit(xs)
        return self
    
    def transform(self, xs):
        return self.transformer.transform(xs)

    def fit_transform(self, xs, y=None):
        return self.transformer.fit_transform(xs)
    
    def inverse_transform(self, xs):
        return self.transformer.inverse_transform(xs)

    def apply_constraints(
        self, 
        xs: jax.Array, 
        cfs: jax.Array, 
        hard: bool = False, 
        rng_key: jax.random.PRNGKey = None, 
        **kwargs
    ):
        return cfs
    
    def compute_reg_loss(self, xs, cfs, hard: bool = False):
        return 0.
    
    def from_dict(self, params: dict):
        self.name = params["name"]
        self.transformer.from_dict(params["transformer"])
        return self
    
    def to_dict(self) -> dict:
        return {"name": self.name, "transformer": self.transformer.to_dict()}